# 一、处理缺失数据

In [1]:
#导包
import numpy as np
import pandas as pd 
import matplotlib as plt

In [2]:
#pandas使用浮点值NaN（Not a Number）表示缺失数据
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
#（1）
#判断是否为空isnull()
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

一些关于缺失值数据处理的函数  
- dropna：根据标签中的值是否存在缺失数据对轴标签进行过滤，可通过阈值调节对缺失值的容忍度；
- fillna：用指定值或插值方法(如ffill或bfill)填充缺失数据；
- notnull：isnull的否定式；

***

## 滤除缺失值（dropna）

In [4]:
#Series
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])  
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [5]:
#DataFrame
#你可能希望丢弃全NA或含有NA的行或列,dropna默认丢弃任何含有缺失值的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [6]:
#过滤后
cleaned

,0,1,2
0,1.0,6.5,3.0


In [7]:
#传入how='all'将只丢弃全为NA的那些行(即将第2行删除)
data.dropna(how='all')
#若要对列操作，则只需传入axis=1即可
#data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


***

## 填充缺失值（fillna）

In [8]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4,1]=NA
df.iloc[:2,2]=NA
df

,0,1,2
0,-0.136382,NaN,NaN
1,-0.449459,NaN,NaN
2,0.552883,NaN,-0.598878
3,0.752979,NaN,-0.278782
4,0.087659,-0.853407,0.587281
5,-0.163387,1.513880,-0.663884
6,-0.499029,-0.542016,0.388330


In [9]:
#自动填充（行列都填充）
df.fillna(123)

,0,1,2
0,-0.136382,123.000000,123.000000
1,-0.449459,123.000000,123.000000
2,0.552883,123.000000,-0.598878
3,0.752979,123.000000,-0.278782
4,0.087659,-0.853407,0.587281
5,-0.163387,1.513880,-0.663884
6,-0.499029,-0.542016,0.388330


In [10]:
#填充列
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.136382,0.500000,0.000000
1,-0.449459,0.500000,0.000000
2,0.552883,0.500000,-0.598878
3,0.752979,0.500000,-0.278782
4,0.087659,-0.853407,0.587281
5,-0.163387,1.513880,-0.663884
6,-0.499029,-0.542016,0.388330


***

# 二、数据转换

## 移除重复的数据（duplicated、drop_duplicates）

In [11]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [12]:
#duplicated方法
#返回一个布尔型Series，表示各行是否是重复行
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [13]:
#drop_duplicates方法
#它会返回一个DataFrame，重复的数组会标为False
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [14]:
#移除指定列的重复值
data.drop_duplicates(['k2'])

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [15]:
#默认是保留第一次出现的，可以改成保留最后一次出现的
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


***

## 利用函数或映射进行数据转换

In [16]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                    'Pastrami', 'corned beef', 'Bacon',
                    'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [17]:
meat_to_animal = {
 'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'
}
#因为meat_to_animal中的key是小写的，所以要将food小写
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [18]:
#利用map将其对应
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


***

## 替换值（replace）

In [19]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [20]:
#将-999替换成NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [21]:
#将数组替换成NaN
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [22]:
#也可以每个值替换不同的值
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [23]:
#传入的参数也可以是字典
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

***

## 离散化和面元划分（cut）

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组。

In [24]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的cut函数。

In [25]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats
#说明：20在(18,25]之内

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [26]:
#面元计数value_counts
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [27]:
#修改区间闭端right=False进行修改
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

***

## 检测和过滤异常值

In [28]:
#过滤或变换异常值（outlier）在很大程度上就是运用数组运算。
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()
#描述

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.059556,0.002271,-0.059922,-0.010490
std,1.013511,1.014860,0.987486,0.975567
min,-2.918786,-3.295933,-3.257440,-3.567071
25%,-0.616161,-0.695274,-0.701063,-0.628602
50%,0.066244,-0.043795,-0.081647,-0.003957
75%,0.757807,0.742673,0.616817,0.648931
max,2.987486,3.956435,3.025008,2.856100


In [29]:
#找出某列中绝对值大小超过3的值
col = data[2]
col[np.abs(col) > 3]

321    3.025008
453   -3.257440
Name: 2, dtype: float64

In [30]:
#选出全部含有“超过3或－3的值”的行，any方法
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
294,-0.777482,-2.051441,-1.082292,-3.380150
321,0.113754,1.448189,3.025008,0.726322
348,-1.867198,-3.199822,-0.087623,0.155894
400,-1.349504,-3.081740,-1.014570,-1.268039
453,2.152362,-0.156310,-3.257440,0.114784
488,-1.024496,-3.295933,-1.082101,-0.442911
679,0.332306,3.956435,-0.185827,1.612290
702,-0.400354,-0.016754,0.062798,-3.567071


In [31]:
#np.sign():根据数据的值是正还是负，np.sign(data)可以生成1和-1
np.sign(data).head()
#head()显示前五行

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,1.0,-1.0,1.0,-1.0
2,-1.0,1.0,-1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,-1.0,-1.0,1.0,-1.0


***

## 排列和随机采样（numpy.random.permutation）

利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。  
通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [32]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [33]:
#随机排列
sampler = np.random.permutation(5)
sampler

array([3, 1, 2, 4, 0])

In [34]:
#基于列的排序
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


***

## 计算指标/哑变量（get_dummies）

&emsp;&emsp;另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。   
&emsp;&emsp;如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。

In [38]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [39]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


***

# 三、字符串操作

- count：返回子串在字符串中出现的次数；
- endswith、startswith：如果字符串以某个后缀结尾（以某个前缀开头），则返回True；
- join:将字符串用作连接其他字符串序列的分隔符；
- index：如果在字符串中找到子串，则返回子串第一个字符所在的位置。如果没有找到，则引发ValueError；
- find：如果在字符串中找到子串，则返回子串第一个字符所在的位置。如果没有找到，则返回-1；
- rfind：如果在字符串中找到子串，则返回子串最后一个字符所在的位置。如果没有找到，则返回-1；
- replace：用另一个字符串替换指定子串；
- strip、rstrip、lstrip：去除空白符（包括换行符）。相当于对各个元素执行x.strip();
- split:通过指定的分隔符将字符串拆分成为一组子串；
- lower、upper：分别将字母字符转换成小写或大写；
- ljust、rjust：用空格（或其他字符）填充字符串的空白侧以返回符合最低宽度的字符串；